<a href="https://colab.research.google.com/github/vranade-cyber/11785_Part-2/blob/main/DL2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#These libraries help to interact with the operating system and the runtime environment respectively
import os
import sys
import time

#Model/Training related libraries
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#Dataloader libraries
from torch.utils.data import DataLoader, Dataset


In [ ]:
#Intall Kaggle API and create kaggle directory

!pip install --upgrade pip
!pip install kaggle==1.5.6

!mkdir .kaggle

#This data is used to login  into your Kaggle account
import json
token = {"username":"virajranade","key":"152c772ecb15eebcb68eb988aaf4e2fd"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [ ]:
import json
token = {"username":"virajranade","key":"152c772ecb15eebcb68eb988aaf4e2fd"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [ ]:

!chmod 600 /content/.kaggle/kaggle.json
!cp /content/.kaggle/kaggle.json /root/.kaggle/
import json
token = {"username":"virajranade","key":"152c772ecb15eebcb68eb988aaf4e2fd"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
!kaggle config set -n path -v /content


In [ ]:
!kaggle competitions download -c idl-fall21-hw2p2s1-face-classification
!kaggle competitions download -c idl-fall21-hw2p2s2-face-verification

In [ ]:
!unzip /content/competitions/idl-fall21-hw2p2s1-face-classification/idl-fall21-hw2p2s1-face-classification.zip
#!unzip /content/test_data.zip
!unzip /content/competitions/idl-fall21-hw2p2s2-face-verification/idl-fall21-hw2p2s2-face-verification.zip

In [ ]:
import os
import numpy as np
from PIL import Image

import torch
import torchvision   
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

In [ ]:
imageFolder_dataset = torchvision.datasets.ImageFolder(root='/content/train_data', 
                                                      transform=(torchvision.transforms.Compose([
                                                      torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                                                      torchvision.transforms.RandomHorizontalFlip(0.5),torchvision.transforms.ToTensor()
                                                      ])))


In [ ]:
imageFolder_dataloader = DataLoader(imageFolder_dataset, batch_size=32, shuffle=True, num_workers=4)

In [ ]:
train_dataset = torchvision.datasets.ImageFolder(root='/content/train_data',transform=(torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                                      torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                                                      torchvision.transforms.RandomHorizontalFlip(0.25)
                                                      ])))
#transform=torchvision.transforms.ToTensor()
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=72, 
                                               shuffle=True, num_workers=4)

map_dict=train_dataset.class_to_idx

dev_dataset = torchvision.datasets.ImageFolder(root='/content/val_data',transform=(torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                                      torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                                                      torchvision.transforms.RandomHorizontalFlip(0.25)
                                                      ])))
#transform=torchvision.transforms.ToTensor()
dev_dataloader = torch.utils.data.DataLoader(dev_dataset, batch_size=72, 
                                             shuffle=False, num_workers=4)

In [ ]:
class block(nn.Module):
    def __init__(
        self, in_channels, intermediate_channels, identity_downsample=None, stride=1
    ):
        super(block, self).__init__()
        self.expansion = 4
        self.conv1 = nn.Conv2d(
            in_channels, intermediate_channels, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn1 = nn.BatchNorm2d(intermediate_channels)
        self.conv2 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False
        )
        self.bn2 = nn.BatchNorm2d(intermediate_channels)
        self.conv3 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels * self.expansion,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False
        )
        self.bn3 = nn.BatchNorm2d(intermediate_channels * self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x):
        identity = x.clone()

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x

In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, layers, image_channels, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        #self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Essentially the entire ResNet architecture are in these 4 lines below
        self.layer1 = self._make_layer(
            block, layers[0], intermediate_channels=64, stride=1
        )
        self.layer2 = self._make_layer(
            block, layers[1], intermediate_channels=128, stride=2
        )
        self.layer3 = self._make_layer(
            block, layers[2], intermediate_channels=256, stride=2
        )
        self.layer4 = self._make_layer(
            block, layers[3], intermediate_channels=512, stride=2
        )

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * 4, num_classes)
        self.dropout= nn.Dropout(0.5)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        #x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        #print(x.shape)
        embed_out=x
        x=self.dropout(x)
        x = self.fc(x)

        return x,x

    def _make_layer(self, block, num_residual_blocks, intermediate_channels, stride):
        identity_downsample = None
        layers = []

        # Either if we half the input space for ex, 56x56 -> 28x28 (stride=2), or channels changes
        # we need to adapt the Identity (skip connection) so it will be able to be added
        # to the layer that's ahead
        if stride != 1 or self.in_channels != intermediate_channels * 4:
            identity_downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels,
                    intermediate_channels * 4,
                    kernel_size=1,
                    stride=stride,
                    bias=False
                ),
                nn.BatchNorm2d(intermediate_channels * 4),
            )

        layers.append(
            block(self.in_channels, intermediate_channels, identity_downsample, stride)
        )

        # The expansion size is always 4 for ResNet 50,101,152
        self.in_channels = intermediate_channels * 4

        # For example for first resnet layer: 256 will be mapped to 64 as intermediate layer,
        # then finally back to 256. Hence no identity downsample is needed, since stride = 1,
        # and also same amount of channels.
        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, intermediate_channels))

        return nn.Sequential(*layers)

In [ ]:
# This is the simplest possible residual block, with only one CNN layer.
# Looking at the paper, you can extend this block to have more layers, bottleneck, grouped convs (from shufflenet), etc.
# Or even look at more recent papers like resnext, regnet, resnest, senet, etc.
class SimpleResidualBlock(nn.Module):
    def __init__(self, in_channel_size,out_channel_size,kernel_size=3, stride=1):
        self.stride=stride
        super().__init__()

        self.in_channel_size=in_channel_size
        self.out_channel_size=out_channel_size

        self.conv1 = nn.Conv2d(in_channel_size, out_channel_size, kernel_size=3, stride=stride,padding=1 ,bias=False)
        self.bn1 = nn.BatchNorm2d(out_channel_size)
        self.conv2=nn.Conv2d(out_channel_size,out_channel_size,kernel_size=3,stride=1,padding=1,bias=False)
        self.bn2=nn.BatchNorm2d(out_channel_size)

        #self.conv3=nn.Conv2d(channel_size,channel_size*2,kernel_size=3,stride=1,padding=1,bias=False)
        #self.bn3=nn.BatchNorm2d(channel_size*2)

        if self.stride == 1:
            self.s = nn.Identity()
        else:
            self.s = nn.Sequential(nn.Conv2d(self.in_channel_size, self.out_channel_size, kernel_size=3, stride=self.stride,padding=1),
                                  nn.BatchNorm2d(self.out_channel_size))
        

    def forward(self, x):

        self.relu = nn.ReLU()
        

        out = self.conv1(x)
        out = self.bn1(out)
        out=self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        '''
        out=self.relu(out)

        out=self.conv3(out)
        out=self.bn3(out)
        '''
        #self.s = nn.Conv2d(self.channel_size, self.channel_size, kernel_size=3, stride=self.stride)
        shortcut = self.s(x)
        #print(shortcut.shape)
        
        self.out = self.relu(out + shortcut)
        
        return self.out

In [ ]:
class ClassificationNetwork(nn.Module):
    def __init__(self, in_features,num_classes,feat_dim = 2):
        super().__init__()
        
        self.layers = nn.Sequential(
            nn.Conv2d(in_features, 64, kernel_size=3, stride=2, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            SimpleResidualBlock(64,64,2),
            SimpleResidualBlock(64,64),
            SimpleResidualBlock(64,64),

            SimpleResidualBlock(64,128,3,2),
            SimpleResidualBlock(128,128),
            SimpleResidualBlock(128,128),
            SimpleResidualBlock(128,128),


            SimpleResidualBlock(128,256,3,2),
            SimpleResidualBlock(256,256),
            SimpleResidualBlock(256,256),
            SimpleResidualBlock(256,256),
            SimpleResidualBlock(256,256),
            SimpleResidualBlock(256,256),
            SimpleResidualBlock(256,256),

            SimpleResidualBlock(256,512,3,2),
            SimpleResidualBlock(512,512),
            SimpleResidualBlock(512,512),

            nn.AdaptiveAvgPool2d((1, 1)), # For each channel, collapses (averages) the entire feature map (height & width) to 1x1
            nn.Flatten(), # the above ends up with batch_size x 64 x 1 x 1, flatten to batch_size x 64
        )

        self.linear = nn.Linear(512, feat_dim)
        self.dropout=nn.Dropout(0.5)
        self.relu = nn.ReLU(inplace=True)
        self.linear_output = nn.Linear(512,num_classes)
        

    def forward(self, x, return_embedding=False):
        embedding = self.layers(x) 
        embedding_out =self.linear(embedding)
        embedding_out=self.dropout(embedding_out)
        embedding_out= self.relu(embedding_out)
        output = self.linear_output(embedding)
        if return_embedding:
            return output,output
        else:
            return output  

In [ ]:
def ResNet50(img_channel, num_classes):
    return ResNet(block, [3, 4, 6, 3], img_channel, num_classes)

In [ ]:
torch.cuda.empty_cache()

numEpochs = 20
in_features = 3 # RGB channels
image_channels=3

learningRate = 0.05
weightDecay = 5e-5

feat_dim=2
#num_classes = len(train_dataset.classes)
num_classes = 4000

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
network = ResNet(block,[3,4,6,3],image_channels,num_classes)
network_34= ClassificationNetwork(in_features,num_classes,feat_dim)
#torch.cuda.empty_cache()
network = network.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(network.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)
#scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,[5,10])
scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode='max',factor=0.5,patience=2)
torch.cuda.empty_cache()
print(network)

Save 2 Models 
1 as a Resnet 50
Other as Resnet 34.
Note-They require different definition.



In [ ]:
best_model_loc = '/content/drive/MyDrive/Latest.pth'
checkpoint = torch.load(best_model_loc)
network.load_state_dict(checkpoint['model_state_dict'])
print(checkpoint['epoch'])


In [ ]:
best_model_loc = '/content/drive/MyDrive/mymodel_34.pth'
checkpoint = torch.load(best_model_loc)
network_34.load_state_dict(checkpoint['model_state_dict'])
print(checkpoint['epoch'])

In [ ]:
print(network)

Each epoch takes around 26 mins for Resnet 50 and 12 to 14 mins for Resnet 34
Train both for 50 to 70 Epochs


In [ ]:
learningRate=1e-2
for epoch in range(50):
    torch.cuda.empty_cache()
    
    optimizer = torch.optim.SGD(network.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)

    # Train
    network.train()
    avg_loss = 0.0
    for batch_num, (x, y) in enumerate(train_dataloader):
        optimizer.zero_grad()
        network=network.to(device)
        x, y = x.to(device), y.to(device)

        outputs = network(x)

        loss = criterion(outputs[0], y.long())
        loss.backward()
        optimizer.step()

        avg_loss += loss.item()

        if batch_num % 100 == 99:
            print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch, batch_num+1, avg_loss/50))
            
            avg_loss = 0.0
        filename = str(f"mynetwork_epoch_{epoch}.pth")

    # Validate
    network.eval()
    num_correct = 0
    for batch_num, (x, y) in enumerate(dev_dataloader):
        x, y = x.to(device), y.to(device)
        outputs,_ = network(x)
        #print(outputs)
        num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()
    scheduler.step(num_correct / len(dev_dataset))
    if (epoch+1)%2==0:
        path=f'/content/drive/MyDrive/Colab_Notebooks/Network_models/mynetwork_epoch_{epoch}.pth'
        torch.save({'epoch': epoch,
        'model_state_dict': network.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict' : scheduler.state_dict()}, path)

        
    print('Epoch: {}, Validation Accuracy: {:.4f}'.format(epoch, num_correct / len(dev_dataset)))

In [ ]:
def parse_data(datadir):
    img_list = []
    ID_list = []
    for root, directories, filenames in os.walk(datadir):
        print(root)  #root: median/1
        for filename in filenames:
            
            if filename.endswith('.jpg'):
                #print(filename)
                filei = os.path.join(root, filename)
                img_list.append(filei)
                ID_list.append(root.split('/')[-1])

    # construct a dictionary, where key and value correspond to ID and target
    uniqueID_list = sorted(list(set(ID_list)))
    class_n = (len(uniqueID_list))
    target_dict = (dict(zip(uniqueID_list, range(class_n))))
    label_list = [target_dict[ID_key] for ID_key in ID_list]

    print('{}\t\t{}\n{}\t\t{}'.format('#Images', '#Labels', len(img_list), len(set(label_list))))
    return img_list, label_list, class_n
    #return img_list,uniqueID_list, class_n

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, file_list, target_list):
        self.file_list = file_list
        self.target_list = target_list
        self.n_class = len(list(set(target_list)))

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img = Image.open(self.file_list[index])
        img = torchvision.transforms.ToTensor()(img)
        label = self.target_list[index]
        return img,label

In [ ]:
def test_model(test_loader,network):
    network.eval()
    predictions=[]
    with torch.no_grad():
        #for batch_num, (x, y) in enumerate(test_dataloader):
        for i in range(8000):
          img = Image.open(f'/content/test_data/{i}.jpg')
          #img = torchvision.transforms.ToTensor()(img) # ()
          transform=torchvision.transforms.Compose([torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                                                    torchvision.transforms.ToTensor()
                                                      ])
          img = transform(img)
          img = torch.as_tensor(img).unsqueeze(0) # (1, 3, 64, 64)
          img=img.to(device)
          predicted_labels=network(img)
          print(i)
          #predicted_labels=pred.argmax(1)
          #loss=criterion(pred,y)
          #test_loss +=loss.item()
          #correct +=(pred.argmax(1)==y).type(torch.float).sum().item()
          predicted_labels=predicted_labels.detach()
          predicted_labels=predicted_labels.cpu()
          predicted_labels=predicted_labels.numpy()
          predicted_labels = np.argmax(predicted_labels, axis=1)
          predicted_labels= reverse_map_dict[predicted_labels[0]]
          predicted_labels = np.reshape(predicted_labels,(-1,1))
          predictions.append(predicted_labels)
        #test_loss /= num_batches
        #correct /= size
        
        predictions=np.vstack(predictions)
        #print(f'\n Test Error is ',correct*100,'Accuracy', test_loss,'Average Loss')
        return (predictions)

In [ ]:
pred = test_model(test_dataloader, network)
np.shape(pred)
predictions = list(np.concatenate(pred).flat)
ids = np.arange(len(predictions))
len(ids)
print(len(predictions))

In [ ]:
idx=np.arange((len(pred)))
print(len(idx))
ids=[]
for i in idx:
  ids.append(str(i)+'.jpg')
preds=[int(i) for i in pred]

In [ ]:
import pandas as pd
my_submission = pd.DataFrame({'id': ids, 'label': preds})
my_submission.to_csv('try9.csv', index=False)


In [ ]:
key1=[]
key2=[]
o=0
with open('/content/verification_pairs_test.txt') as f:
    for line in f:
        o=o+1
        keys=line.split(' ')
        #print(keys[0])
        #print(keys[1])
        p1=keys[0].split('/')
        p2=keys[1].split('/')



        key1.append(p1[1])
        key2.append(p2[1])

    #keys[1]
    #print(len(key1))
    #print(o)
    print(key1)

In [ ]:
def parse_data(datadir):
    img_list = []
    ID_list = []
    for root, directories, filenames in os.walk(datadir):  #root: median/1
        for filename in filenames:
            if filename.endswith('.jpg'):
                filei = os.path.join(root, filename)
                img_list.append(filei)
                ID_list.append(filename)
    return img_list, ID_list

In [ ]:
class Test_Dataset2(Dataset):
    def __init__(self, file_list,target_list):
        self.file_list = file_list
        self.target_list=target_list
        assert len(self.file_list)==len(self.target_list)

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img = Image.open(self.file_list[index])
        img = torchvision.transforms.ToTensor()(img)
        img = torchvision.transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])(img)
        #print(img)
        label = self.target_list[index]
        return img,label

In [ ]:
img_list,ID_list=parse_data('/content/test_data')
print(ID_list)
print(len(img_list))
ver_set=Test_Dataset2(img_list,ID_list)

In [ ]:
img1 = Image.open(img_list[0])
img1 = torchvision.transforms.ToTensor()(img1)
img1 = torchvision.transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])(img1)

In [ ]:
cos=nn.CosineSimilarity(eps=1e-6)

ver=[]
ver34=[]
id=[]



for i in len(key1):
    o1=key1[i]
    o2=key2[i]

    img1 = Image.open(img_list[o1])
    img1 = torchvision.transforms.ToTensor()(img1)
    img1 = torchvision.transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])(img1)

    img2 = Image.open(img_list[o2])
    img2 = torchvision.transforms.ToTensor()(img2)
    img2 = torchvision.transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])(img2)
    
    _,x1=network(img1)
    _,x2=network(img2)

    _,y1=network_34(img1)
    _,y2=network_34(img2)

    C1=cos(x1,x2)
    C2=cos(y1,y2)

    ver.append(C1)
    ver34.append(C2)

    k='/verification_data/'+str(o1)+'.jpg'+'/verification_data/'+str(o2)+'.jpg'
    id.append(k)





In [ ]:
import pandas as pd

my_submission = pd.DataFrame({'Id': id, 'Category': ver})
my_submission.to_csv('try10.csv', index=False)

In [ ]:
a=0.93

Ensemble=ver34*a+ ver*(1-a)
my_submission = pd.DataFrame({'Id': id, 'Category': Ensemble})
my_submission.to_csv('try11.csv', index=False)